# Title

This Notebook...

## Initialization

### Imports

In [ ]:
import numpy
import matplotlib.pyplot
import re
import ast

import sys
sys.path.append("../")
import congestionfinder
import congestionfinder.bpsdetector
import congestionfinder.road
import congestionfinder.detection
import congestionfinder.speedflow
import congestionfinder.congestion
import patchfinder.patch

In [ ]:
import logging
import io
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
consoleHandler = logging.StreamHandler()
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)

class PlotFilter(logging.Filter):
    def filter(self, record):
        return "Found patches:" in record.getMessage()

log_capture_string = io.StringIO()
variableHandler = logging.StreamHandler(log_capture_string)
variableHandler.addFilter(PlotFilter())
logger.addHandler(variableHandler)

## Classes

In [ ]:
bpsDetector = congestionfinder.bpsdetector.BPSDetector("00D00C03405B18200005")
print(bpsDetector)

In [ ]:
road = congestionfinder.road.Road(12)
road.addBPSDetector(bpsDetector)
print(road)
road.indexDetectorSpaces()
print(road.getSpaceToSpaceIndex())

In [ ]:
detection = congestionfinder.detection.Detection("00D00C03405B18200005", 1234, 1440, 120, 17)
print(detection)

## Roads

In [ ]:
roads = congestionfinder.road.parseBPSCodesToRoads(["10D00204D037D007000B", "10D002051800D0070013", "10D002057000D007000F"])
for value in roads.values():
    value.indexDetectorSpaces()

In [ ]:
for value in sorted(roads.values(), key = lambda x:len(x.getBPSDetectors())):
    print(value)

## Speed and Flows

In [ ]:
detections = []
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 1, 80, 17))
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 2, 70, 16))
detections.append(congestionfinder.detection.Detection("10D00204D037D007000B", 30855, 3, 75, 17))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 1, 40, 10))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 2, 55, 18))
detections.append(congestionfinder.detection.Detection("10D002051800D0070013", 32600, 3, 35, 12))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 1, 70, 19))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 2, 85, 14))
detections.append(congestionfinder.detection.Detection("10D002057000D007000F", 34800, 3, 100, 19))

In [ ]:
speeds, flows, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex = congestionfinder.speedflow.parseDetectionsToSpeedsAndFlows(detections, roads[2])

In [ ]:
print(speeds.shape)
print(flows.shape)
print(minSpaceIndex)
print(maxSpaceIndex)
print(minTimeIndex)
print(maxTimeIndex)
matplotlib.pyplot.imshow(speeds, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flows, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsHighFlow, flowsHighFlow, maskHighFlow = congestionfinder.speedflow.removeLowFlowTimes(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsHighFlow.shape)
print(flowsHighFlow.shape)
print(sum(maskHighFlow) / (sum(maskHighFlow) + sum(~maskHighFlow)))
matplotlib.pyplot.imshow(speedsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsHighFlow, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
speedsWorkingDetectors, flowsWorkingDetectors, maskWorkingDetectors = congestionfinder.speedflow.removeMissingDetectors(speeds, flows) ## Do something with Mask

In [ ]:
print(speedsWorkingDetectors.shape)
print(flowsWorkingDetectors.shape)
print(sum(maskWorkingDetectors) / (sum(maskWorkingDetectors) + sum(~maskWorkingDetectors)))
matplotlib.pyplot.imshow(speedsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()
matplotlib.pyplot.imshow(flowsWorkingDetectors, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion

In [ ]:
speedThreshold = 65
flowThreshold = 40
congestions = congestionfinder.congestion.parseSpeedFlowsToCongestions(speedsWorkingDetectors, flowsWorkingDetectors, speedThreshold, flowThreshold)

In [ ]:
print(congestions.shape)
matplotlib.pyplot.imshow(congestions, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
congestions[1, 1] = numpy.nan
congestionsWithoutMissingValues = congestionfinder.congestion.interpolateMissingValues(congestions)

In [ ]:
print(congestionsWithoutMissingValues.shape)
matplotlib.pyplot.imshow(congestionsWithoutMissingValues, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

In [ ]:
spaceSmoothing = 1
timeSmoothing = 2
congestionsSmoothed = congestionfinder.congestion.applySmoothingFilter(congestionsWithoutMissingValues, spaceSmoothing, timeSmoothing)

In [ ]:
print(congestionsSmoothed.shape)
matplotlib.pyplot.imshow(congestionsSmoothed, aspect = "auto")
matplotlib.pyplot.colorbar()
matplotlib.pyplot.show()

## Congestion Filter

In [ ]:
testDataArray = numpy.random.rand(16,9)
testBooleanArray = testDataArray >= 0.9
print(testBooleanArray)

In [ ]:
patches = patchfinder.patch.findPatches(testBooleanArray)

In [ ]:
matches = re.findall("\[\d*, \d*, \d*, \d*\]", log_capture_string.getvalue())
for match in matches:
    array = ast.literal_eval(match)
    patch = patchfinder.patch.Patch(array[0], array[1], array[2], array[3])
    congestionfinder.congestion.plotCongestionsWithPatches(testBooleanArray, [patch])

In [ ]:
for patch in patches:
    print(patch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(testDataArray, patches)
congestionfinder.congestion.plotCongestionsWithPatches(testBooleanArray, patches)

In [ ]:
patchesFiltered = congestionfinder.congestion.filterLargeCongestions(patches, 1)

In [ ]:
for patch in patchesFiltered:
    print(patch)

In [ ]:
marginSpace = 1
marginTime = 1
patchesWithMargins = congestionfinder.congestion.addMargins(patchesFiltered, marginSpace, marginTime, minSpaceIndex, maxSpaceIndex, minTimeIndex, maxTimeIndex)

In [ ]:
for patch in patchesWithMargins:
    print(patch)

In [ ]:
congestionfinder.congestion.plotCongestionsWithPatches(testDataArray, patchesWithMargins)
congestionfinder.congestion.plotCongestionsWithPatches(testBooleanArray, patches)

End of Notebook